In [16]:
import numpy as np
import pandas as pd
import glob
import matplotlib

In [17]:
file_path = "/Users/oladotun/Documents/MEST Program/SEP 769/Deep Learning Projects/Project 1/archive_copy/"

In [20]:
normal_files = glob.glob(file_path + "normal/**/*.csv", recursive=True)
imbalance_fault_files = glob.glob(file_path + "imbalance/**/*.csv", recursive=True)
#print(imbalance_fault_files)

In [43]:
def dataReader(files_array, output_value = 0):
    df = pd.DataFrame()
    for file in files_array:
        df_temp = pd.read_csv(file,header=None)
        df = pd.concat([df,df_temp],ignore_index=True)
    df[len(df.columns)] = output_value
    return df

In [44]:
df_normal = dataReader(normal_files, 0)
df_imbalance = dataReader(imbalance_fault_files, 1)

In [45]:
df_normal

,0,1,2,3,4,5,6,7,8
0,4.55950,0.175200,0.287210,-0.017751,-0.41565,0.032459,-0.11218,-0.128140,0
1,4.60380,-0.051295,-0.194050,-0.060071,-0.41809,0.036547,-0.11043,0.118310,0
2,4.57030,-0.969080,0.038033,-0.028329,-0.43081,0.041924,-0.14331,-0.071527,0
3,4.58700,0.891270,0.072973,0.007453,-0.40017,0.041090,-0.11984,0.043445,0
4,4.58870,-1.716000,-0.329290,-0.033063,-0.50281,0.040474,-0.25270,0.023901,0
...,...,...,...,...,...,...,...,...,...
499995,-0.57644,0.807760,-0.021390,-0.036499,0.60025,0.042079,0.26363,0.111130,0
499996,-0.59596,-1.996800,-0.028970,-0.029717,0.55223,0.044877,0.21696,-0.023022,0
499997,-0.62033,1.628100,0.245700,0.039413,0.62443,0.044588,0.28238,0.020543,0
499998,-0.61151,-2.253600,-0.351000,-0.011765,0.52908,0.043317,0.20342,0.152240,0


In [46]:
df_imbalance

,0,1,2,3,4,5,6,7,8
0,-0.64234,-0.46940,0.019808,0.033536,0.12485,0.055209,1.52160,-0.098177,1
1,-0.62009,1.12570,0.126180,0.047859,0.12804,0.057756,1.52390,0.079246,1
2,-0.64030,-1.18030,-0.189380,0.022482,0.12323,0.056203,1.37320,0.096866,1
3,-0.61602,1.39750,0.355200,0.104900,0.13191,0.059919,1.53410,-0.090876,1
4,-0.64451,-0.57308,-0.597810,0.050062,0.12487,0.055433,1.42400,0.086766,1
...,...,...,...,...,...,...,...,...,...
3499995,4.53150,-0.57258,-0.409320,-0.034251,0.16444,0.057461,-0.27455,0.139430,1
3499996,4.48340,-0.54411,0.224580,0.017620,0.16391,0.059489,-0.25040,-0.112630,1
3499997,4.51850,0.51511,-0.114840,-0.020558,0.16625,0.059566,-0.22627,0.101420,1
3499998,4.49840,-1.73250,-0.133580,-0.030462,0.16389,0.058135,-0.27519,-0.009295,1
